In [2]:
import sys
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
print(sys.version)
print(np.__version__)
print(sklearn.__version__)
print(pd.__version__)
print(sns.__version__)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.19.2
0.23.2
1.1.3
0.11.0


In [3]:
import datetime
from pandas_datareader import data
from sklearn.model_selection import train_test_split

In [4]:
companies_dict={
    "Amazon":"AMZN",
    "Apple":"AAPL",
     "Walgreen":"WBA",
    "Northrop Grumman":"NOC",
    "Boeing":"BA",
    "Lockhead Martin":"LMT",
    "Mc Donalds":"MCD",
    "Intel":"INTC",
    "Navistar":"NAV",
    "IBM":"IBM",
    "Texas Instruments":"TXN",
    "MasterCard":"MA",
    "Microsoft":"MSFT",
    "General Electronics":"GE",
    "American Express":"AXP",
    "Pepsi":"PEP",
    "Coca Cola":"KO",
    "Johnson and johnson":"JNJ",
    "Toyota":"TMC",
    "Honda":"HMC",
    "Motsubishi":"MSBHY",
    "SONY":"SNY",
    "Exxon":"XOM",
    "Chevron":"CVX",
    "Valero energy":"VLO",
    "Ford":"F",
    "Bank of America":"BAC"
}

In [5]:
data_source="yahoo"
start_date="2015-01-01"
end_date="2017-12-31"
panel_data=data.DataReader(list(companies_dict.values()),data_source,start_date,end_date)
print(panel_data.axes)

[DatetimeIndex(['2014-12-31', '2015-01-02', '2015-01-05', '2015-01-06',
               '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-12',
               '2015-01-13', '2015-01-14',
               ...
               '2017-12-15', '2017-12-18', '2017-12-19', '2017-12-20',
               '2017-12-21', '2017-12-22', '2017-12-26', '2017-12-27',
               '2017-12-28', '2017-12-29'],
              dtype='datetime64[ns]', name='Date', length=756, freq=None), MultiIndex([('Adj Close',  'AMZN'),
            ('Adj Close',  'AAPL'),
            ('Adj Close',   'WBA'),
            ('Adj Close',   'NOC'),
            ('Adj Close',    'BA'),
            ('Adj Close',   'LMT'),
            ('Adj Close',   'MCD'),
            ('Adj Close',  'INTC'),
            ('Adj Close',   'NAV'),
            ('Adj Close',   'IBM'),
            ...
            (   'Volume',   'JNJ'),
            (   'Volume',   'TMC'),
            (   'Volume',   'HMC'),
            (   'Volume', 'MSBHY'),
            ( 

In [6]:
stock_close=panel_data["Close"]
stock_open=panel_data["Open"]
print(stock_close.iloc[0])

Symbols
AMZN       310.350006
AAPL        27.594999
WBA         76.199997
NOC        147.389999
BA         129.979996
LMT        192.570007
MCD         93.699997
INTC        36.290001
NAV         33.480000
IBM        160.440002
TXN         53.470001
MA          86.160004
MSFT        46.450001
GE          24.298077
AXP         93.040001
PEP         94.559998
KO          42.220001
JNJ        104.570000
TMC      11500.000000
HMC         29.520000
MSBHY       36.759998
SNY         45.610001
XOM         92.449997
CVX        112.180000
VLO         49.500000
F           15.500000
BAC         17.889999
Name: 2014-12-31 00:00:00, dtype: float64


In [7]:
stock_close=np.array(stock_close).T
stock_open=np.array(stock_open).T
print(stock_close.shape)

(27, 756)


In [8]:
row,col=stock_close.shape
movements=np.zeros([row,col])
for i in range(row):
    movements[i,:]=np.subtract(stock_close[i,:],stock_open[i,:])

print(movements.shape)

In [9]:
print(movements.shape)
print(movements)

(27, 756)
[[-1.19998169e+00 -4.05999756e+00 -4.82000732e+00 ...  2.34997559e+00
  -2.90002441e+00 -1.28800049e+01]
 [-6.10000610e-01 -5.14999390e-01 -5.10000229e-01 ...  1.25000000e-01
   2.00004578e-02 -3.22502136e-01]
 [-2.90000916e-01 -1.10000610e-01 -5.80001831e-01 ... -4.70001221e-01
   6.00051880e-02 -9.99450684e-03]
 ...
 [-6.00013733e-02  7.90000916e-01 -1.56999969e+00 ...  3.19999695e-01
  -8.99963379e-02 -7.79998779e-01]
 [-1.00002289e-02 -2.30000496e-01 -3.59999657e-01 ... -6.99996948e-02
   1.00000381e-01 -9.00001526e-02]
 [-3.00001144e-01 -9.00001526e-02 -4.10001755e-01 ...  6.99996948e-02
   6.99996948e-02 -3.29999924e-01]]


In [10]:
from sklearn.preprocessing import Normalizer
from numpy import *
normalizer=Normalizer()
z=isnan(movements)
movements[z]=0
new=normalizer.fit_transform(movements)
print(new.max())
print(new.min())

0.3597736430289109
-0.32190273323639396


In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=10,max_iter=10000)
pipeline=make_pipeline(normalizer,kmeans)

In [12]:
pipeline.fit(movements)

Pipeline(steps=[('normalizer', Normalizer()),
                ('kmeans', KMeans(max_iter=10000, n_clusters=10))])

In [13]:
print(kmeans.inertia_)

9.459355514316215


In [25]:
labels=pipeline.predict(movements)
print(companies_dict)
companies=sorted(companies_dict.items(),key=lambda x: x[0])
df=pd.DataFrame({"labels":labels,"companies":companies})

{'Amazon': 'AMZN', 'Apple': 'AAPL', 'Walgreen': 'WBA', 'Northrop Grumman': 'NOC', 'Boeing': 'BA', 'Lockhead Martin': 'LMT', 'Mc Donalds': 'MCD', 'Intel': 'INTC', 'Navistar': 'NAV', 'IBM': 'IBM', 'Texas Instruments': 'TXN', 'MasterCard': 'MA', 'Microsoft': 'MSFT', 'General Electronics': 'GE', 'American Express': 'AXP', 'Pepsi': 'PEP', 'Coca Cola': 'KO', 'Johnson and johnson': 'JNJ', 'Toyota': 'TMC', 'Honda': 'HMC', 'Motsubishi': 'MSBHY', 'SONY': 'SNY', 'Exxon': 'XOM', 'Chevron': 'CVX', 'Valero energy': 'VLO', 'Ford': 'F', 'Bank of America': 'BAC'}


In [26]:
print(df.sort_values('labels'))

    labels                   companies
13       0  (Johnson and johnson, JNJ)
19       0             (Navistar, NAV)
14       0      (Lockhead Martin, LMT)
25       0        (Valero energy, VLO)
8        0                   (Ford, F)
26       0             (Walgreen, WBA)
4        1                (Boeing, BA)
3        1      (Bank of America, BAC)
5        1              (Chevron, CVX)
0        2              (Amazon, AMZN)
1        2     (American Express, AXP)
11       2                  (IBM, IBM)
12       2               (Intel, INTC)
21       3                (Pepsi, PEP)
17       3           (Microsoft, MSFT)
6        3             (Coca Cola, KO)
15       3            (MasterCard, MA)
16       3           (Mc Donalds, MCD)
2        4               (Apple, AAPL)
10       5                (Honda, HMC)
9        5   (General Electronics, GE)
7        5                (Exxon, XOM)
18       6         (Motsubishi, MSBHY)
22       7                 (SONY, SNY)
23       7    (Texas Inst

In [28]:
from sklearn.decomposition import PCA
reduced_data=PCA(n_components=2).fit_transform(new)
kmeans=KMeans(n_clusters=10)
kmeans.fit(reduced_data)
labels=kmeans.predict(reduced_data)

In [29]:
df=pd.DataFrame({"labels":labels,"companies":companies})
print(df.sort_values('labels'))

    labels                   companies
24       0               (Toyota, TMC)
19       0             (Navistar, NAV)
9        0   (General Electronics, GE)
14       0      (Lockhead Martin, LMT)
0        1              (Amazon, AMZN)
11       1                  (IBM, IBM)
12       1               (Intel, INTC)
3        2      (Bank of America, BAC)
5        2              (Chevron, CVX)
16       2           (Mc Donalds, MCD)
15       2            (MasterCard, MA)
23       3    (Texas Instruments, TXN)
22       3                 (SONY, SNY)
18       4         (Motsubishi, MSBHY)
26       5             (Walgreen, WBA)
8        5                   (Ford, F)
25       5        (Valero energy, VLO)
21       6                (Pepsi, PEP)
4        6                (Boeing, BA)
2        6               (Apple, AAPL)
10       7                (Honda, HMC)
7        7                (Exxon, XOM)
1        7     (American Express, AXP)
20       8     (Northrop Grumman, NOC)
13       8  (Johnson and 